В современном мире машинного обучения тема **объяснимости и интерпретируемости** моделей становится неотъемлемой частью развития искусственного интеллекта.

Когда алгоритмы машинного обучения используются для принятия решений, особенно в чувствительных областях, таких как медицина или право, а также в научных применениях, важно понимать, почему модель делает тот или иной вывод.

Мы рассмотрим концепции и методы, касающиеся способов представления и объяснения выводов, полученных от моделей машинного обучения.



# Мотивация использования Explainability

Модели машинного обучения представляют собой черный ящик.

<center><img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/input_blackbox_output.png" alt="alttext" width="400"/></center>

Иногда это  становится препятствием для внедрения моделей.

Есть как минимум три причины, по которым нас может интересовать объяснение предсказаний модели.

**1. Доверие к предсказаниям**

Нельзя остановить ядерную электростанцию или назначить пациенту опасное лечение лишь на основании предсказания «черного ящика».
Даже маловероятная ошибка в таких случаях будет иметь тяжелые последствия. Поэтому человек, принимающий решение, должен понимать, на основе каких признаков или симптомов сделано предсказание.

**2. Обнаружение некорректных зависимостей**

Модель может использовать совсем не те признаки, которые соответствуют реальным объектам. То есть вместо реальных свойств объекта ориентироваться на случайно обнаруженные зависимости, не связанные с реальными данными (фон, водяной знак и т. п.).

Яркий пример мы можем увидеть в статье ["Why Should I Trust You?" Explaining the Predictions of Any Classifier](https://arxiv.org/abs/1602.04938), авторы которой обучили классификатор волков и хаски на изображениях, отобранных так, чтобы на всех фотографиях волков на фоне был снег, а на фотографиях хаски — нет. На рисунке ниже мы можем увидеть, на что обращает внимание при предсказаниях полученная в результате такого обучения модель.

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L14/2_bad_models_prediction.png" alt="alttext" width="400"/></center>

**3. Публикации в научных журналах**

Вероятность публикации статьи значительно повышается, если автор смог объяснить происхождение результатов своего исследования.

<font size="5"> Explainability & Interpretability </font>

<center><img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/explainability_vs_interpretability.png" alt="alttext" width="1000"/></center>


В англоязычной литературе можно встретить два термина, связанных с темой доверия к ML-моделям: Explainability и Interpretability. Терминология в этой области еще не устоялась, и в разных источниках эти термины определяют по-разному. Можно выделить следующие характерные различия:

* **Explainability** — методики, позволяющие объяснить механизм функционирования модели. Например, для линейной регрессии это анализ коэффициентов при параметрах.

* **Interpretability** — анализ того, как изменение входов модели влияет на ее выходы. Например, закрашивая часть пикселей изображения, можно выяснить, какие из них повлияют на предсказания (пример с хаски и волками).

Мы рассмотрим оба типа методов.

[Machine Learning Explainability vs Interpretability: Two concepts that could help restore trust in AI](https://www.kdnuggets.com/2018/12/machine-learning-explainability-interpretability-ai.html)


# Объяснимость моделей классического ML

Некоторые алгоритмы машинного обучения обладают свойством объяснимости сами по себе. Объяснить, почему модель выдала то или иное предсказание, возможно непосредственно исходя из понимания принципа работы модели.

Чтобы подчеркнуть, что, в отличие от "черных ящиков", внутреннее устройство таких моделей прозрачно и понятно, их называют "белыми" или "прозрачными" ящиками (glass box).

В частности, к таким алгоритмам относятся **линейные модели** и **модели, основанные на деревьях решений**.

<center><img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/glass_box.png" alt="alttext" width="400"/></center>


## Оценка важности признаков для линейных моделей


Выход линейной модели является линейной комбинацией входных признаков, к которой также добавляется смещение:

$$ \large
\begin{eqnarray*}
y & = & w_1 x_1+ w_2 x_2 + ... + w_n x_n + b\\
& = & \sum_{i=1}^n x_i w_i + b \\
& = & \left(\vec{x}, \vec{w}\right) + b
\end{eqnarray*}
$$

Данную формулу можно графически представить следующим образом:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/one_neuron_linear_model.png" width="450"></center>

В случае с **линейными моделями** легко определить, какие признаки модель считает важными. Если модель присваивает какому-то признаку **большой вес**, то этот признак сильнее влияет на **предсказание**.

При этом стоит отметить, что **признаки должны быть сравнимы**, то есть должны находиться в сравнимых диапазонах и быть выражены в одних единицах измерения (или быть безразмерными).


### Пример для табличных данных (Boston Dataset)

Для примера скачаем **датасет жилья Бостона** ([The Boston Housing Dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)), в котором проанализируем зависимость цены на жилье от параметров жилья и района, в котором оно находится.

In [ ]:
import pandas as pd

# load dataset
boston_dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/boston_dataset.csv",
    index_col=0,
)
x_data = boston_dataset.iloc[:, :-1]
y_data = boston_dataset["target"].values

boston_dataset.head()

Посмотрим на статистики признаков:

In [ ]:
boston_dataset.describe()

Из описательных статистик по признакам видно, что признаки несравнимы. Поэтому коэффициенты линейной модели, обученной на таких данных, будут не только нести в себе важность, но и масштабировать значения, чтобы перевести их в размерность целевой переменной. Этот факт будет мешать интерпретировать значения весов перед признаками как степень их влияния на предсказание целевой переменной.


Обучим модель на **стандартизованных данных**:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

ss = StandardScaler()
x_data_scaled = ss.fit_transform(x_data)

model = LinearRegression()
model.fit(x_data_scaled, y_data)

Выведем **коэффициенты** признаков, отсортированные по абсолютному значению:

In [ ]:
linear_model_coefs = model.coef_
feature_names = x_data.columns

linear_importance = pd.DataFrame(
    {"name": feature_names, "coef": linear_model_coefs}
).sort_values("coef", key=abs, ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

linear_importance["sign"] = linear_importance["coef"].apply(
    lambda x: "neg" if x < 0 else "pos"
)
palette = {"neg": "#1e88e5", "pos": "#ff0d57"}

plt.figure(figsize=(5, 5))
plt.title("Linear model coefficients")
ax = sns.barplot(
    data=linear_importance,
    y="name",
    x="coef",
    hue="sign",
    palette=palette,
    legend=False,
    orient="h",
)
for i in ax.containers:
    ax.bar_label(i, fmt="%.2f", label_type="center")
plt.tight_layout()
plt.show()

Добавим абсолютные значения для сравнения с другими оценками важности.

In [ ]:
linear_importance["abs(coef)"] = linear_importance["coef"].abs()

## Оценка важности признаков для деревьев решений

В случае с **деревьями** всё далеко не так очевидно: дерево не знает такой концепции, как **"вес признака"**.

Универсального критерия значимости для деревьев у нас нет, и, в зависимости от задачи и от того, как эти признаки устроены, ответы могут быть разными.

Один из способов оценки качества признаков для дерева решений следует из принципа построения дерева. При выборе признака и порога для разбиения выборки в узле дерева используется метрика $\text{Gini}$:

$$\large \text{Gini} = 1 - \sum_ip_i^2,$$

где $p_i$ — вероятность того, что объект, попавший в данный  лист, относится к $i$-му классу. Чем меньше $\text{Gini}$ в листьях, тем лучше узел, от которого "растут" листья, разделяет классы.


<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.3/L03/out/compute_gini_for_binary_features.png" width="900"></center>

Для того, чтобы охарактеризовать, насколько данный узел хорошо разделяет выборку, $\text{Gini}$ листьев данного узла суммируются с весами, равными доле объектов, попавших в каждый из листев (выражение в скобках в формуле ниже). Далее данный показатель вычитается из $\text{Gini}$ узла разбиения. Таким образом формируется показатель $\text{impurity decrease}$, который характеризует, как хорошо данное разбиение "очищает" выборки (то есть делает подвыборки в листьях более однородными по целевой переменной).

$$\large \text{Impurity decrease} = \text{Gini}_0 - \left(\dfrac{n_1}{n_1+n_2}\text{Gini}_1 + \dfrac{n_2}{n_1+n_2}\text{Gini}_2\right),$$

где $n_1, n_2$ — число объектов в листьях, $ \text{Gini}_0$ — чистота исходного узла.

**Боль в груди:**

 $\text{Impurity decrease} = 0.498 - (\dfrac{138}{138+159})\cdot 0.364 - (\dfrac{159}{138+159})\cdot 0.336 = 0.149$

**Хорошо циркулирует кровь:**

 $\text{Impurity decrease} = 0.498 - (\dfrac{164}{164+133})\cdot 0.349 - (\dfrac{133}{164+133})\cdot 0.373 = 0.138$

**Есть атеросклероз:**  

$\text{Impurity decrease} = 0.498 - (\dfrac{123}{123+174})\cdot 0.377 - (\dfrac{174}{123+174})\cdot 0.383 = 0.117$

Наибольший $\text{impurity decrease}$ в признаке "боль в груди". Значит, мы возьмем "боль в груди" как признак, на основании которого продолжим строить дерево.  

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.3/L03/out/compute_gini_for_another_features.png" width="600"></center>

Именно $\text{impurity decrease}$ используется для расчета атрибута [️️`feature_importances_`](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html) в Sklearn. Для случайного леса (и других ансамблей) просто выдается среднее по деревьям.


У данного метода есть **недостаток:** он может завышать качество признаков с большим количеством возможных значений.

Даже если признак не информативен, но у него много значений, на нем можно выбрать большое количество порогов, по которым можно разбить данные, что приведет к переобучению.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.3/L03/out/decision_tree_for_real_numbers.png" width="1000"></center>

Бинарные и категориальные признаки с небольшим числом категорий потенциально могут получать заниженное качество по сравнению с вещественными, даже если те дают худшее разбиение.

### Пример для табличных данных (Boston Dataset)

Посмотрим на важность признаков, основанную на **impurity decrease**.

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

rng = np.random.RandomState(42)
model = RandomForestRegressor(random_state=rng)
model.fit(x_data, y_data)

In [ ]:
rf_model_importances = model.feature_importances_

gini_importance = pd.DataFrame(
    {
        "name": feature_names,
        "feature importances": rf_model_importances,
    }
).sort_values("feature importances", ascending=False)

In [ ]:
def importances_diagram(data, x, y, title, fmt="%.2f"):
    plt.title(title)
    ax = sns.barplot(
        data=data,
        y=y,
        x=x,
        color=sns.xkcd_rgb["azure"],
        orient="h",
    )
    for i in ax.containers:
        ax.bar_label(i, fmt=fmt, label_type="edge")

In [ ]:
plt.figure(figsize=(13, 4))
plt.subplot(1, 2, 1)
importances_diagram(
    data=linear_importance,
    x="abs(coef)",
    y="name",
    title="Linear model importances",
)

plt.subplot(1, 2, 2)
importances_diagram(
    data=gini_importance,
    x="feature importances",
    y="name",
    title="Random forest importances",
)
plt.show()

Можно видеть, что важность признаков для одних и тех же данных зависит от модели. При этом признаки `RM` — количество комнат в доме, и `LSTAT` — процент людей с низким социальным статусом (без среднего образования, безработных), важны для обеих моделей, что достаточно логично.

In [ ]:
num_unique = x_data.agg(lambda x: len(np.unique(x)))
num_unique.name = "num unique"
num_unique = pd.DataFrame(num_unique)
num_unique["name"] = num_unique.index
gini_importance = gini_importance.merge(num_unique, how="inner", on="name")

Посмотрим на соотношение между диаграммами количества уникальных значений у признаков и важностей признаков.

In [ ]:
plt.figure(figsize=(13, 4))
plt.subplot(1, 2, 1)
importances_diagram(
    data=gini_importance,
    x="num unique",
    y="name",
    title="Number of unique values in features",
    fmt="%d",
)

plt.subplot(1, 2, 2)
importances_diagram(
    data=gini_importance,
    x="feature importances",
    y="name",
    title="Random forest importances",
)
plt.show()

Интересно отметить, что признаки `LSTAT`, `RM`, `DIS` и `CRIM`, оказавшиеся наиболее важными для Random Forest, имеют большое количество уникальных значений. В то же время признаки `AGE` и `B` также имеют большое количество уникальных значений, хотя их важность для модели случайного леса не высока.

# Методы, изучающие отклик модели на изменение входных данных

В этом разделе мы рассмотрим методы, изучающие отклик модели на изменение входных данных. Они также называются **Model-Agnostic Methods** и находятся ближе всего к концепции “черного ящика”. Эти методы изучают связь между входами и выходами модели и пытаются ее объяснить.

<center><img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/black_box_interpretability.png" alt="alttext" width="500"/></center>

## ICE (Individual Conditional Expectation)

Одним из самых простых и интуитивно понятных методов является метод Individual Conditional Expectation (ICE). Он заключается в следующем:
1. Обучаем и **фиксируем модель**.
2. Выбираем **один объект**.
3. Выбираем **один признак** этого объекта, который мы будем **менять в некотором диапазоне**, все остальные признаки фиксируем.
4. Меняем этот признак и смотрим, как меняется **предсказание модели**.
5. Строим кривую зависимости **предсказанного значения** от **изменяемого признака** для модели.

Если применить метод ICE на множестве объектов, а затем усреднить результаты, то получится метод, который называется Partial Dependence (PD).

Посмотрим, как этот метод работает на примере модели Random Forest.

### Пример для табличных данных (Boston Dataset)

Загрузим и предобработаем  данные:

In [ ]:
import pandas as pd

# load dataset
boston_dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/boston_dataset.csv",
    index_col=0,
)
x_data = boston_dataset.iloc[:, :-1]
y_data = boston_dataset["target"].values

boston_dataset.head()

Обучим модель:

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

rng = np.random.RandomState(42)
model = RandomForestRegressor(random_state=rng)
model.fit(x_data, y_data)

Нам интересно посмотреть на два признака, которые были наиболее важны для предсказания: **`RM`** — среднее количество комнат в жилье, **`LSTAT`** — процент людей с низким социальным статусом, и на один признак, который для  **Random Forest** не важен: **`AGE`** — возраст постройки.

В Sklearn реализован класс `PartialDependenceDisplay` [[doc]🛠️](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.PartialDependenceDisplay.html), который реализует методы ICE и PD.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

_, ax = plt.subplots(ncols=3, figsize=(15, 5), sharey=True, constrained_layout=True)

features_info = {
    "features": ["RM", "LSTAT", "AGE"],  # features to inspect
    "kind": "both",  # plot both individual lines (ICE) and average line (PD)
}

common_params = {
    "subsample": 50,  # how many samples to draw
    "n_jobs": 2,  # for parallelizing
    "grid_resolution": 20,  # how many steps of feature changing on x-axis
    "random_state": 0,  # seed for random sampling
}

x_data = pd.DataFrame(x_data, columns=x_data.columns)

display = PartialDependenceDisplay.from_estimator(
    model,
    x_data,
    **features_info,
    ax=ax,
    **common_params,
)

Синие линии — это ICE для отдельных объектов. По оси $x$ — изменяемые признаки для этих объектов, по оси $y$ — изменение целевого значения. Оранжевым цветом нарисовано среднее по объектам (PD).

Видно, что увеличение количества комнат в большинстве случаев положительно влияет на цену жилья, увеличение процента людей с низким социальным статусом — отрицательно, а возраст постройки не важен.


## LIME (Local Interpretable Model-agnostic Explanations)

### Принцип работы

Нам бы хотелось оценивать все признаки одновременно для любой модели. Для этого можно попробовать **заменить "черный ящик"** (black-box) **"прозрачным"** (glass-box).

Ключевая идея [**LIME**](https://arxiv.org/abs/1602.04938) — **локальная аппроксимация сложно-интерпретируемой** (black-box) модели при помощи **легко-интерпретируемой** (glass-box).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/lime_idea.png" width="400"></center>

Давайте разбирать идею по частям:
1. **Локальная аппроксимация** — значит, что мы берем **один объект** и **модифицируем** его (изменяем признаки), чтобы получить небольшой датасет, локализованный вокруг исходного объекта.
2. **Сложно интерпретируемая модель** — модель, для которой мы проводим оценку (наш "черный ящик"). Ее мы используем для того, чтобы получить **предсказания** для датасета, построенного на основе одного объекта.
3. Таким образом, мы получаем **датасет**, включающий **признаки** и **предсказания** “черного ящика”. На этом датасете **учим** “прозрачный ящик” — **легко-интерпретируемую модель**, для которой мы умеем определять важность признаков. Например, линейную модель или дерево решений.

Идею **LIME** можно проиллюстрировать следующим образом:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/lime_example.png" width="1000"></center>



По **осям** отложены значения **двух непрерывных признаков**. Изначально мы имеем сложно интерпретируюмую модель, которая разделяет пространство признаков на две области со сложной границей.

**Шаг 1.** Выбираем один объект, для которого мы хотим получить локальное объяснение (на иллюстрации это $\large x^*$).

**Шаг 2.** Семплируем вокруг интересущего нас объекта новые объекты (черные точки).

**Шаг 3.** Используем исходную модель для разметки семплированных точек — получаем новый датасет, представляющий локальную область вокруг интересующего нас объекта.

**Шаг 4.** Взвешиваем объекты из локального датасета с учетом их расстояния до интересующего нас объекта.

**Шаг 5.** Обучаем на полученном локальном датасете линейную модель с учетом весов для новых объектов. В локальной области интересующего нас объекта линейная модель ведет себя схоже с исходной сложной моделью. Используем важность признаков простой линейной модели как оценку важностей признаков для сложной модели в локальной области интересующего нас объекта.


### Как получить набор объектов вблизи искомого?

* В **текстах** можно **удалить слово**.
* Для **изображений** можно **делить картинку на области** (суперпиксели) и поочередно закрашивать их одним и тем же цветом (средним).
* Для **табличных данных**:
 * для **бинарных** переменных в низкоразмерном пространстве достаточно просто менять значение переменной на противоположное (**0 на 1** и **1 на 0**),
 * для **вещественных переменных** были предложены различные варианты. Например, к ним можно прибавлять **Гауссов шум** или **дискретизировать** (например, по квантилям).

### Ограничения

Описанный подход позволяет интерпретировать поведение модели **только в некоторой области** вблизи интересующего нас объекта.

На практике этого может быть достаточно. **Работает быстро**, так как не требует перебора всех комбинаций признаков.

### Пример для табличных данных (Boston Dataset)

Загрузим и предобработаем  данные

In [ ]:
import pandas as pd

# load dataset
boston_dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/boston_dataset.csv",
    index_col=0,
)
x_data = boston_dataset.iloc[:, :-1].values
y_data = boston_dataset["target"].values

boston_dataset.head()

Обучим модель

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

rng = np.random.RandomState(42)
model = RandomForestRegressor(random_state=rng)
model.fit(x_data, y_data)

In [ ]:
!pip install -q lime

Применим Lime

In [ ]:
from lime.lime_tabular import LimeTabularExplainer


explainer = LimeTabularExplainer(training_data=x_data,
                                 training_labels=y_data,
                                 mode="regression",
                                 feature_names = boston_dataset.columns.tolist())

In [ ]:
explanation = explainer.explain_instance(x_data[42, :],
                                         predict_fn=model.predict)

explanation.show_in_notebook(show_table=True)

### Пример NLP (классификация статей)

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.3/L14/christian_or_atheist.png" alt="alttext" width="900"/></center>

Используем датасет [fetch_20newsgroups](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html).

Данные [«The 20 Newsgroups»](http://qwone.com/~jason/20Newsgroups/) — это коллекция примерно из **20&nbsp;000 новостных документов**, разделенная (приблизительно) равномерно между **20 различными категориями**.

Коллекция «The 20 Newsgroups» стала популярным набором данных для экспериментов с техниками машинного обучения для текстовых приложений, таких как классификация и кластеризация.

В данном примере мы будем использовать [Multinomial Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) для классификации и TF-IDF для представления текстов.

In [ ]:
import sklearn
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")
# making class names shorter
class_names = [
    x.split(".")[-1] if "misc" not in x else ".".join(x.split(".")[-2:])
    for x in newsgroups_train.target_names
]
class_names[3] = "pc.hardware"
class_names[4] = "mac.hardware"

for i, class_name in enumerate(class_names):
    print(f"{i:<3d}{class_name}")

Мы берем **два класса**, которые трудно различить, потому что в них много схожих слов: **христианство и атеизм**.

Обучая модель, мы получаем **точность на тестовых данных 83,5%**, что является удивительно высоким показателем. Если бы точность была нашим единственным мерилом доверия, мы бы точно доверились этому классификатору.

In [ ]:
import sklearn.metrics
from sklearn.naive_bayes import MultinomialNB

# Again, let's use the tfidf vectorizer, commonly used for text.
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

# Train the model
model_nb = MultinomialNB(alpha=0.01)
model_nb.fit(train_vectors, newsgroups_train.target)

# Calculate F1_score
pred = model_nb.predict(test_vectors)

f1_metric = sklearn.metrics.f1_score(newsgroups_test.target, pred, average="weighted")

print(f"f1-score on test: {f1_metric:.3f}")

Мы видим, что этот классификатор имеет очень высокий F1 score. [Руководство Sklearn для 20 newsgroups](https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes) указывает, что **Multinomial Naive Bayes переобучается** на этом наборе данных, изучая нерелевантные взаимосвязи, такие как заголовки.

Теперь мы используем **LIME** для объяснения отдельных предсказаний.

Как видно из кода, текст подается на вход модели не в сыром виде, а после предобработки объектом `vectorizer`.

Сперва нам потребуется создать обертку — пайплайн, который объединяет `vectorizer` и модель:

[`sklearn.pipeline.make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html)

In [ ]:
from sklearn.pipeline import make_pipeline

model_with_preprocessing = make_pipeline(vectorizer, model_nb)

[`LimeTextExplainer`](https://lime-ml.readthedocs.io/en/latest/lime.html#lime.lime_text.LimeTextExplainer) имеет метод `explain_instance` с такими аргументами:


```python
explain_instance(
    text_instance,
    classifier_fn,
    labels=(1, ),
    top_labels=None,
    num_features=10,
    num_samples=5000,
    distance_metric="cosine",
    model_regressor=None)
```

**`classifier_fn`** *— функция для классификации, которая получает список из `d` строк и выдает `(d, k)` numpy-массив с предсказанными вероятностями для `k` классов.*

В нашем случае такой функцией будет `model_with_preprocessing.predict_proba`.

Давайте посмотрим на LIME-объяснение для произвольного экземпляра в тестовом наборе.

В случае многоклассовой классификации мы должны определить, для каких меток хотим получить объяснения, с помощью параметра `labels`. **Сгенерируем пояснения** для меток 0 и 15:




In [ ]:
!pip install -q lime

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer


explainer = LimeTextExplainer(class_names=class_names, random_state=42)
idx = 1340
explanation = explainer.explain_instance(
    newsgroups_test.data[idx],
    model_with_preprocessing.predict_proba,
    num_features=10,
    labels=[0, 15],
)

Возвращается специальный объект класса [`Explanation`](https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=Explanation#lime.explanation.Explanation):

In [ ]:
print(explanation.as_list(label=0))
print(explanation.as_list(label=15))

Обратите внимание, что положительный и отрицательный знаки относятся к конкретной метке, так что слова, отрицательные по отношению к классу 0, могут быть положительными по отношению к классу 15, и наоборот.

Теперь давайте посмотрим на **визуализацию объяснений**.
Обратите внимание на то, что для каждого класса слова в правой части строки являются «положительными», а слова в левой части — «отрицательными» для объясняемого класса.

Также видно, что в классификаторе используются как **осмысленные слова** (такие как «Theism», «Genocide» и т. д.), так и **неосмысленные** (название университета «Rice», домен «owlnet»).

In [ ]:
explanation.show_in_notebook(text=newsgroups_test.data[idx], labels=(0,))

In [ ]:
explanation.show_in_notebook(text=newsgroups_test.data[idx], labels=(15,))

На этом примере можно увидеть, что в заголовке или кавычках может быть и полезный сигнал, который будет помогать обобщению (например,  в строке «Тема»).

А есть и **слова, которые нельзя обобщать** (например, адреса электронной почты и названия учреждений).

## SHAP (SHapley Additive exPlanations)

###Принцип работы

<center><img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/shap_scheme.png" alt="alttext" width="600"/></center>


Цель [**SHAP** 📚[book]](https://christophm.github.io/interpretable-ml-book/shap.html) — объяснить предсказание объекта $x$ путем **вычисления вклада каждого признака** в предсказание. Для этого вычисляются SHAP-значения, основанные на **значениях Шепли** из **теории игр**.

**SHAP** рассматривает объясняемую **модель как игру**, а **признаки** объектов — как **коалицию игроков**. **SHAP-значения** говорят нам, как **справедливо распределить "выигрыш" между игроками** — это вклад, который каждый признак вносит в предсказание модели.





### Свойства чисел Шепли

Числа Шепли, разработанные Ллойдом Шепли в 1953 году, являются важным понятием в теории кооперативных игр. Они справедливо распределяют общий выигрыш коалиции игроков, учитывая индивидуальные вклады каждого участника. Числа Шепли определяют вклад каждого игрока, анализируя все возможные коалиции и вычисляя средний предельный вклад каждого игрока.

Числа Шепли обладают рядом свойств, которые делают их эффективными и справедливыми. Перечислим некоторые из них:

1. **Эффективность**: сумма чисел Шепли всех игроков равна общему выигрышу коалиции. Это означает, что весь выигрыш распределяется без остатка.

2. **Нулевая игра**: игрок, не вносящий дополнительного вклада ни в одну коалицию, получает нулевое вознаграждение. Это означает, что ненужные или неактивные участники не получают долю выигрыша.

3. **Симметрия**: если два игрока вносят одинаковый вклад во все возможные коалиции, они получают одинаковое вознаграждение. Это свойство обеспечивает справедливое распределение в случае равного вклада.

### Теоретико-игровой пример

Рассмотрим на примере, как оценивается вклад игроков в общий выигрыш с помощью чисел Шепли.

Пусть у нас есть уличная музыкальная группа из трех участников: барабанщика, гитариста и солиста. Они выступают на улице и собирают деньги в шляпу. Музыканты хотят оценить, **какой вклад в заработок привносит каждый участник группы**.

Для этого можно устроить эксперимент: несколько вечеров подряд группа будет выступать для публики **в разных комбинациях состава участников** (на иллюстрации все комбинации изображены в виде вершин графа и пронумерованы, ребра графа обозначают добавление участника в состав):
* пустое множество участников (пустая шляпа на улице, вершина $1$),
* все участники по одному (вершины $2$–$4$),
* все участники по двое (вершины $5$–$7$),
* полный состав из трех музыкантов (вершина $8$).

<center>
<img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/shap_features_graph.png" alt="alttext" width="550"/>
</center>

Для простоты представим, что каждый вечер тот или иной состав группы
оценивается одной и той же публикой. Оценкой является заработок за вечер. Музыканты поиграли по такой схеме все восемь вечеров и зафиксировали заработок за каждый вечер.

<center>
<img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/shap_predictions_graph.png" alt="alttext" width="550"/>
</center>

Теперь в каждой вершине графа находится оценка того или иного состава группы публикой.

Зная оценки всех возможных комбинаций состава этой группы, мы можем посчитать вклад каждого участника в итоговый заработок.

Вклад участника в заработок вычисляется на основании так называемых **дополнительных вкладов** *(marginal contribution, MC)*.

**Дополнительный вклад игрока** — это **разница** между заработком состава, **включающим данного игрока**, и заработком состава **без данного игрока**.

В данном случае можно рассчитать дополнительный вклад для игрока как разницу оценок вершин, соединенных ребром в графе.

Например, дополнительный вклад барабанщика по сравнению с пустой группой
$\{∅\}$ рассчитывается следующим образом (см. вершины $2$ и $1$):


$$ \large \text{MC}_{🥁,\{∅\}}= \text{Earning}_{\{🥁\}}\ - \text{Earning}_{\{∅\}}=  4\ 000 - 0 = 4\ 000$$

А дополнительный вклад барабанщика при добавлении его в пару к солисту будет равен (см. вершины $6$ и $4$):

$$ \large \text{MC}_{🥁,\{🎤\}}= \text{Earning}_{\{🥁🎤\}}\ \ - \text{Earning}_{\{🎤\}}\ =  8\ 500 - 10\ 000 = -\ 1\ 500$$

Отрицательный дополнительный вклад возможен и означает, что добавление учасника в группу снижает заработок (может быть, барабанщик играет невпопад и без него солист получает больше, чем с ним).

Для того, чтобы **оценить итоговый вклад барабанщика** в оценку всей группы публикой, нужно учесть его **дополнительные вклады** во все комбинации, **где он участвовал** (в графе выделены соответствующие вершины):

<center>
<img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/shap_estimation_important_features.png" alt="alttext" width="550"/>
</center>

SHAP-значение для барабанщика в этой группе является общим вкладом игрока в заработок и вычисляется как **взвешенная сумма его дополнительных вкладов**:

$$ \large \text{SHAP}_{🥁}=w_1\cdot \text{MC}_{🥁,\{\emptyset\}}+w_2\cdot \text{MC}_{🥁,\{🎸\}}+w_3\cdot \text{MC}_{🥁,\{🎤\}}+w_4\cdot \text{MC}_{🥁,\{🎸🎤\}} $$

Веса определяются согласно правилу: сумма весов дополнительных вкладов для каждого уровня графа должна быть равной 1.

Таким образом, нетрудно рассчитать веса дополнительных вкладов барабанщика:
* первый уровень содержит $3$ ребра, каждое из которых будет иметь вес $\dfrac{1}{3}$
* второй уровень содержит $6$ ребер, каждое из которых будет иметь вес $\dfrac{1}{6}$
* третий уровень содержит $3$ ребра, каждое из которых будет иметь вес $\dfrac{1}{3}$

Таким образом: $\displaystyle w_1=w_4=\frac{1}{3}, \; w_2=w_3=\frac{1}{6}$.

**В этой аналогии:**
* конкретный музыкальный коллектив — это один объект выборки,
* музыканты — это признаки,
* публика — это модель,
* заработок группы (оценка группы публикой) — предсказание модели (оценка объекта моделью).

Однако эта аналогия в случае моделей машинного обучения требует дополнительного пояснения. В примере с музыкальной группой мы использовали одну модель (публику) для оценки разных комбинаций признаков (разных составов группы).

Отсутствие какого-то игрока в группе означает отсуствие признака в признаковом описании объекта. Большинство моделей ML не могут работать с пропущенными значениями (то есть с признаковыми описаниями различной длины). Поэтому для получения оценок придется всякий раз **обучать модель заново на новом подмножестве признаков** (сохраняя гиперпараметры модели и набор обучающих объектов).

Для получения предсказания модели для пустого множества признаков может быть использовано, например, среднее значение целевой переменной по обучающей выборке.




### Kernel SHAP

У описанного выше метода есть недостаток: нам нужно обучить огромное количество моделей. Для всего десяти признаков это $2^{10} = 1024$ модели.

На помощь нам приходит статья [A Unified Approach to Interpreting Model Predictions](https://arxiv.org/pdf/1705.07874.pdf). В ней введено понятие **аддитивного метода атрибуции признаков**, в котором в качестве "стеклянного ящика" используется функция бинарных переменных (есть признак/нет признака):

$$\large g(z') = \phi_0 +\sum_{i=1}^M\phi_i z_i',$$

где $z'\in\{0,1\}^M$ — вектор из $0$ и $1$ длины $M$: $0$ — отсутствие признака, $1$ — наличие признака, $M$ — количество упрощенных признаков (например, суперпикселей для изображения), $\phi_i$ — вклад (важность) $i$-го упрощенного признака.

Для адекватного описания работы модели данный метод должен удовлетворять трем свойствам. Эти свойства позволяют методу SHAP обеспечивать интерпретируемость и справедливость оценки важностей признаков для моделей машинного обучения, подобно тому, как числа Шепли обеспечивают справедливое распределение выигрыша в теории игр.

1. **Local Accuracy** (локальная точность, аналог свойства "Эффективность" для чисел Шепли) — результат модели объяснения $g(x’)$ должен совпадать с результатом оригинальной модели $f(x)$ (вклад всех признаков суммируется в значение предсказания модели):

$$\large f(x)=g(x')=\phi_0 +\sum_{i=1}^M\phi_i x_i',$$

где:
* $x' = \{x_1', x_2', ...,x_M'\}$ — упрощенные признаки объекта (например, суперпиксели для изображения),
*  $x$ — оригинальный набор признаков объекта (например, значения RGB пикселей изображения),
* упрощенный набор признаков переводится в оригинальный с помощью функции $h_x(x)$:

$$\large x=h_x(x')$$


2. **Missingness (отсутствие влияния отсутствующих признаков, аналог свойства "Нулевая игра" для чисел Шепли)** — если признак отсутствует ($x_i = \text{None}$ означает, что упрощенный признак $x_i' = 0$), он вносит нулевой вклад:

$$\large x_i' = 0 \Rightarrow \phi_i = 0$$

Это свойство означает, что признаки, не влияющие на предсказание модели, имеют нулевые значения SHAP.

3. **Consistency (согласованность)** — если для двух моделей $f'$ и $f$ устранение $i$-го признака изменяет предсказание первой модели больше, чем второй, то и SHAP-значение $\phi_i$ для этого признака должно быть больше для первой модели. Формально это свойство выражается таким образом:

> Пусть $f_x(z')=f(h_x(z'))$ и $z' \text{\\} i$ обозначает установку $z_i'=0$. Для двух любых моделей $f$ и $f'$, **если**

> $$\large f'_x(z') - f'_x(z'\text{\\}i) \geq f_x(z') - f_x(z'\text{\\}i)$$

> для всех входов $z'\in\{0,1\}^M$, **тогда** $\large \phi_i(f', x) \geq \phi_i(f, x)$.

Это свойство обеспечивает интуитивную согласованность между разными моделями: если $i$-й признак важнее для первой модели, чем для второй, то и его SHAP-значение должно быть больше для первой модели, чем для второй.

Такой метод расчета называется Kernel SHAP. Он позволяет не обучать огромное количество моделей и реализовать вычисление значений Шепли за конечное время. Именно так считаются значения Шепли в библиотеке SHAP.

### Пример для табличных данных (Boston Dataset)

Установим пакет SHAP:

In [ ]:
!pip install -q shap

Для примера скачаем датасет жилья Бостона (`boston_dataset.csv`), в котором проанализируем зависимость цены на жилье от параметров жилья и района, в котором оно находится.

Разобьем данные на обучающие и тестовые и обучим Random Forest.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


# load dataset
boston_dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/boston_dataset.csv",
    index_col=0,
)
x_data = boston_dataset.iloc[:, :-1]
y_data = boston_dataset["target"]

# Split the data into train and test data
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

# Build the model with the random forest regression algorithm
rng = np.random.RandomState(42)
model = RandomForestRegressor(n_jobs=-1, max_depth=4, random_state=rng)
model.fit(x_train, y_train)

Применим SHAP.

In [ ]:
import shap

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
explanations = explainer(x_test)

Вызов `explainer` возвращает объект класса `Explanation`, который хранит:
- данные, для которых получены SHAP-объяснения, в атрибуте `.data`,
- массив SHAP-значений для каждого признака каждого объекта в атрибуте `.values`,
- массив базовых значений для каждого объекта в атрибуте `.base_values`.

In [ ]:
print(f"Type of the explanations object: {type(explanations)}")

print(f"Shape of x_test:                       {x_test.shape}")
print(f"Shape of the explanations data:        {explanations.data.shape}")
print(f"Shape of the explanations SHAP values: {explanations.values.shape}")
print(f"Shape of the explanations base values: {explanations.base_values.shape}\n")

print(
    f"Is explanations.data equal x_test: {(explanations.data == x_test.values).all()}"
)

**Force plot**

Хороший способ визуализировать вклад каждого признака в конкретный прогноз — использовать так называемый "график сил" (force plot).

В приведенном ниже примере показан график сил для первого объекта в тестовом наборе данных.

In [ ]:
# load JS visualization code to notebook
shap.initjs()
# visualize the first prediction’s explanation
shap.plots.force(explanations[0])

* $f(x)$ — это прогноз модели по анализируемому объекту недвижимости. А "base value" — это средний прогноз по всему набору тестовых данных. Или, другими словами, это значение, которое можно было бы спрогнозировать, если бы мы не знали никаких характеристик текущего примера.

* Признаки, которые способствуют увеличению цены, показаны красным, а те, которые уменьшают, — синим.

* Длина стрелки — “сила” влияния. Численное значение (оно не совпадает с длиной) — важность признака. Из нашего анализа мы помним, что увеличение `RM` положительно влияет на предсказание, но для данного объекта значение `RM` влияет на цену отрицательно и, возможно, это повод дополнительно исследовать данный объект.

**Waterfall plot**

Другой способ визуализации SHAP-объяснения для конкретного примера — так называемый "график-водопад" (waterfall plot). Фактически это график сил, где стрелки, обозначающие сдвиг предсказания от базового значения за счет SHAP-значений, расположены одна под другой, начиная с самой длинной сверху.

In [ ]:
# visualize the first prediction's explanation using waterfall

shap.plots.waterfall(explanations[0])

Этот график объясняет движущие силы конкретного прогноза: влияние каждого отдельного признака (наименее значимые признаки объединяются в одну группу) представлено стрелками, которые перемещают предсказание влево и вправо, начиная с базового значения (внизу картинки).


Указанный выше пример приведен только для одного объекта.

Если мы возьмем много объяснений в виде графика сил, повернем их на 90 градусов, а затем сложим их по горизонтали, мы сможем увидеть объяснения для всего набора данных (в блокноте этот график является интерактивным):

In [ ]:
# load JS visualization code to notebook
shap.initjs()
# visualize the training set predictions
shap.force_plot(explanations)

* По оси $x$ — объекты, по оси $y$ — вклад признаков в предсказание для каждого объекта. Важно заметить, что объекты упорядочены по схожести, которая считается по расстоянию между объектами.
* Можно использовать выпадающее меню, чтобы посмотреть, как будет выглядеть график без упорядочивания или при упорядочивании по конкретному признаку.


**Bar plot**

Сводный график `shap.plots.bar` даст нам график важности признаков.

Признаки с высокой предсказательной способностью показаны вверху, а с низкой предсказательной силой — внизу.

In [ ]:
shap.plots.bar(explanations)

Обратите внимание, что, согласно SHAP, наименьшей предсказательной способностью обладают признаки **`CHAS`, `ZN`, `RAD`** и **`INDUS`**.

Здесь мы только что рассмотрели алгоритм `TreeExplainer` для интерпретации модели.

Вы можете изучить остальные алгоритмы: `DeepExplainer`, `KernelExplainer`, `LinearExplainer` и `GradientExplainer`.

### Пример NLP (перевод с английского на русский)

С помощью SHAP можно строить интерпертации предсказаний более сложных, в том числе нейросетевых, моделей.

Рассмотрим пример интерпретации модели для предварительно обученной модели машинного перевода
[Machine Translation Explanations 🐾[git]](https://github.com/slundberg/shap/blob/master/notebooks/text_examples/translation/Machine%20Translation%20Explanations.ipynb). Для перевода будем использовать предобученную [модель-трансформер 🎓[arxiv]](https://arxiv.org/abs/1810.04805).

Используем одну из моделей [Hugging Face 🐾[git]](https://github.com/huggingface/transformers).

Модель: [Language Technology in Helsinki ✏️[blog]](https://blogs.helsinki.fi/language-technology/)

[[doc] 🛠️ Language Technology Research Group at the University of Helsinki](https://huggingface.co/Helsinki-NLP)

[[doc] 🛠️ Helsinki-NLP/opus-mt-en-ru](https://huggingface.co/Helsinki-NLP/opus-mt-en-ru)

In [ ]:
!pip install -q transformers[sentencepiece]

Загружаем модель.

Для этого используется класс `AutoModelForSeq2SeqLM`, на вход которому передается имя модели, а возвращает он объект соответствующего класса.

In [ ]:
import torch
import transformers
from transformers import AutoModelForSeq2SeqLM
from IPython.display import clear_output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lang = "en"
target_lang = "ru"
model_name = f"Helsinki-NLP/opus-mt-{lang}-{target_lang}"

# Download the model and the tokenizer
# can also try translation with different pre-trained models

# It's a Factory pattern
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)
clear_output()
print(type(model))

В данном случае нам вернулся объект типа `MarianMT` [🛠️[doc]](https://huggingface.co/transformers/model_doc/marian.html).



Теперь создадим [токенайзер 🛠️[doc]](https://huggingface.co/transformers/main_classes/tokenizer.html).

Как мы уже обсуждали, токенайзер преобразовывает слова и знаки препинания из исходного текста в токены, которые можно подать на вход модели. В данном случае возвращаются id. При этом не всегда одно слово преобразовывается в один токен, иногда слово разбивается по слогам на несколько токенов.

Токенайзер создается с помощью класса `AutoTokenizer` по имени модели.

In [ ]:
!pip install -q sacremoses

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

clear_output()

print(type(tokenizer))

input = tokenizer("Hello world!", return_tensors="pt")
print(input)

translated = model.generate(
    **tokenizer("Hello world!", return_tensors="pt").to(device), max_new_tokens=512
)
# ** -  is dictionary unpack operator
# https://pavel-karateev.gitbook.io/intermediate-python/sintaksis/args_and_kwargs

Теперь переведем целую фразу и проанализируем, как выход модели связан со входом.

Для этого создадим объект `shap.Explainer` [🛠️[doc]](https://shap.readthedocs.io/en/latest/generated/shap.Explainer.html), который в данном случае инициализируется экземпляром модели и экземпляром токенайзера.



> *В действительности вторым параметром конструктора `shap.Explainer` не обязательно должен быть токенайзер. `shap.Explainer` принимает объект, поддерживающий интерфейс `masker`:*

> ```masked_args = masker(*model_args, mask=mask)```

> *Он используется для исключения части аргументов, и токенайзеры поддерживают этот интерфейс (`shap.TokenMasker`). Благодаря такому подходу SHAP может работать с различными моделями как с "черным ящиком".*





Вместо того, чтобы запускать саму модель, мы запускаем `Explainer` (неявно вызывая его метод `__call__`).


In [ ]:
import shap

# Define the input sentences we want to translate
data = ["SHAP is a method to explain machine learning models predictions."]

# We build an explainer by passing the model we want to explain and
# the tokenizer we want to use to break up the input strings
explainer = shap.Explainer(model, tokenizer, max_new_tokens=512)

# Explainers are callable, just like models
explanation = explainer(data)
clear_output()

На выходе получаем объект класса `shap.Explanation` [🛠️[doc]](https://shap.readthedocs.io/en/latest/generated/shap.Explanation.html#shap-explanation), который содержит значения Шепли для каждого токена.

In [ ]:
print("Data", explanation.data)
print("SHAP values shape", explanation.shape)  # 1, in, out

Теперь, используя интерактивную визуализацию `shap.plots.text` [🛠️[doc]](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/text.html), можно отобразить результат объяснения.

In [ ]:
shap.initjs()
shap.plots.text(explanation)

Данная языковая модель предсказывает эмбеддинги — вектора, которые преобразуются в токены. При этом SHAP для оценки важности использует сжатые представления эмбеддингов. В данном случае наибольший интерес представляет не раскраска `outputs` (абсолютное значение сжатого представления эмбеддинга на выходе), а подсветка `inputs`, которая появляется, когда мы нажимаем на выходной токен. Она показывает, какие входные токены влияют на выходной.

Анализ важностей через SHAP в данном примере похож на анализ карт внимания в трансформерах: можно проследить, какие входные токены наиболее важны для конкретного выходного токена. При этом SHAP дает более наглядную визуализацию, напрямую связывая выход и вход модели. Анализ карт внимания затруднителен в больших моделях, состоящих из многих слоев: сложно проследить полный путь трансформации входных токенов в выходные, так как важности "размываются".

# Заключение

Мы рассмотрели основные методы, которые используются для оценки важности признаков для моделей классического ML. Линейные модели и модели, основанные на деревьях, могут быть объяснены нативно исходя из принципов их построения и работы. В иных случаях могут быть полезны методы, которые пытаются интерпретировать предсказания модели основываясь на изменении входных данных. Среди таких мы рассмотрели LIME и SHAP.

Пренебрежение объяснением того, почему модель дала тот или иной результат, ведет к недоверию по отношению не только к самой модели, но и к конкретным прогнозам, и, следовательно, является существенным препятствием для внедрения вашей модели или публикации статьи в научном журнале.

